# Loading libraries & Setup

Importing Numpy, Matplotlib, Tensorflow 2 and Keras.

In [127]:
import pandas as pd

# NumPy - mathematical functions on multi-dimensional arrays and matrices
import numpy as np

# Matplotlib - plotting library to create graphs and charts
# import matplotlib.pyplot as plt

# TensorFlow - end-to-end open source platform for machine learning.
import tensorflow as tf

# Keras - high-level API for TensorFlow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# OS module for Python
import os

In [ ]:
# def showImages(arr):
#     fig, axes = plt.subplots(1, 5, figsize=(20, 20))
#     axes = axes.flatten()
#     for img, ax in zip(arr, axes):
#         ax.imshow(img)
#     plt.tight_layout()
#     plt.show()

In this project we will use three data sets (images) of cats and dogs.

- Train data set to train and fit our model.
- Validation data set that we will be using during the training of the model to validate the effectiveness of the training.
- Test data set that we will be using to test our model to see hw it performs.

Now let's define where the images for training, validation and test are in our system.

In [162]:
train_dir = './data/train'
validate_dir = './data/validation'
test_dir = './data/test'

train_dogs_dir = os.path.join(train_dir, 'dogs')
train_cats_dir = os.path.join(train_dir, 'cats')
validate_dogs_dir = os.path.join(validate_dir, 'dogs')
validate_cats_dir = os.path.join(validate_dir, 'cats')
test_cats_and_dogs_dir = os.path.join(test_dir, 'cats_and_dogs')

Let's calculate the number of images in each directory that we will later use for the model training.

In [123]:
num_dogs_train = len(os.listdir(train_dogs_dir))
num_cats_train = len(os.listdir(train_cats_dir))
num_dogs_validate = len(os.listdir(validate_dogs_dir))
num_cats_validate = len(os.listdir(validate_cats_dir))

train_total = num_dogs_train + num_cats_train
validate_total = num_dogs_validate + num_cats_validate
test_total = len(os.listdir(test_cats_and_dogs_dir))

We will define the batch size which we will use for our ImageDataGenerator to define how many training examples are utilized in one iteration of training.

We will also define the image size which defines the size of the image our ImageDataGenerator will generate for the training.

In [30]:
batch_size = 32
img_size = 150

Now we will configure our ImageDataGenerator. ImageDataGenerator is Keras function data enables us data augmentation which means replacing the original batch of images with new and randomly transformed batch. This is useful and improves the training of our model because we can feed our model with new (augmented) images in each epoch.

We will define our ImageDataGenerator here only with rescale=1./255 that will standardize the numeric values in the matrix of our images.

You cal also try the following options:
- shear_range=0.2 - defines shear trainsformation intensity to our image
- zoom_range=0.2 - defines the range of random zoom applied to ur images
- horizontal_flip=True - randomly flips images horizontally.

Feel free to experiment more by using the documentation of the function here: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

Remember that adding more options to the ImageDataGenerator adds complexity and therefore increases comsumption of the processing pwoer and the memory so experiment to find the right balance.

In [31]:
img_gen = ImageDataGenerator(rescale=1./255)

Above, we have defined general criteria for our image generator now we will define the flow of images for each data set train, validate and test using flow_from_directory function of ImageDataGenerator.

Here are the configuration option we are using:
 - batch_size - how many training examples are utilized in one iteration of training
 - directory - where our images are
 - shuffle - whether to shuffle the images
 - target_size - the target size of our image that the function will return
 - class_mode - we are using "binary" because in our example we have two categories cats or dogs

In [163]:
train_img_gen = img_gen.flow_from_directory(batch_size=batch_size,
                                           directory=train_dir,
                                           shuffle=True,
                                           target_size=(img_size, img_size),
                                           class_mode='binary')

validate_img_gen = img_gen.flow_from_directory(batch_size=batch_size,
                                               directory=validate_dir,
                                               shuffle=False,
                                               target_size=(img_size, img_size),
                                               class_mode='binary')

test_img_gen = img_gen.flow_from_directory(batch_size=batch_size,
                                               directory=test_dir,
                                               shuffle=False,
                                               target_size=(img_size, img_size),
                                               class_mode=None)

Found 20000 images belonging to 2 classes.
Found 4800 images belonging to 2 classes.
Found 200 images belonging to 1 classes.


Let's display some images generated from our train image generator.

In [ ]:
# showImages([train_img_gen[0][0][0] for i in range(15)])

Now let's create our Neural Network for to distinguish images of cats and dogs.

The model we are going to use for our networks is the sequential model which is suitable for most problems. It does not allow you to create models that share layers or have multiple inputs or outputs.

We will then add to our model a few 2D convolution layers.

We are explaining the meaning of the layers and parameters in the code but here are some of the most important elements:

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
 - 32 means the number of output filters in the convolution
 - (3, 3) is kernel size of the layer which specifies the height and width of the 2D convolution window. It usually depends on the size of the image
 - activation='relu' - defines the activation function on the output of the nodes of the layer. ReLU is the most commonly used activation function in neural networks. You can experiment and learn more about diffeernt activation functions on Keras documentation (https://keras.io/api/layers/activations/).
 - input_shape=(150, 150, 3) is the image size 150x150 with the three RGB colors

model.add(MaxPooling2D(pool_size=(2, 2)))
 - Max pooling operation for 2D spatial data which is a downsampling strategy in Convolutional Neural Networks. It downsamples the input by taking the maximum value over the window defined by pool_size for each dimension.
 
model.add(Flatten())
 - Flattens the input so we can introduce a standard Dense layer that will lead us to a single result layer. Before this operation we have three dimensional data of width, height, and color of each pixel of the image

model.add(Dense(256, activation='relu'))
 - regular densely-connected layer. Densely-connected means that each neuron in a layer receives an input from all the neurons in the previous layer.

model.add(Dropout(0.5))
 - The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time. This helps prevent overfitting.

model.add(Dense(1, activation='sigmoid'))
 - is the last layer in the network which will return the probability of a cat or a dog as a number between 0-cat and 1-dog

In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

Now we need to compile our netowrk with the loss function, optimizer function and we define the metrics as accuracy so we can see how the accuracy of our network is improving during the fitting process.

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', # rmsprop or adam
              metrics=['accuracy'])

We can now view the summary so we can see in details the structure of our neural network model including number and types of layers, total parameters, etc.

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       7

And we can start the model training process using the train_img_gen generator and also validating at each ste using validate_img_gen.

In [10]:
fit_result = model.fit_generator(
            train_img_gen,
            steps_per_epoch=int(np.ceil(train_total / float(batch_size))),
            epochs=5, 
            validation_data=validate_img_gen,
            validation_steps=int(np.ceil(validate_total / float(batch_size)))
            )

Epoch 1/5
626/626 [==============================] - 1210s 2s/step - loss: 0.6438 - accuracy: 0.6274 - val_loss: 0.5164 - val_accuracy: 0.7470
Epoch 2/5
626/626 [==============================] - 1159s 2s/step - loss: 0.4909 - accuracy: 0.7664 - val_loss: 0.4545 - val_accuracy: 0.7892
Epoch 3/5
626/626 [==============================] - 1130s 2s/step - loss: 0.4025 - accuracy: 0.8225 - val_loss: 0.3625 - val_accuracy: 0.8354
Epoch 4/5
626/626 [==============================] - 1131s 2s/step - loss: 0.3429 - accuracy: 0.8529 - val_loss: 0.3830 - val_accuracy: 0.8226
Epoch 5/5
626/626 [==============================] - 1138s 2s/step - loss: 0.3076 - accuracy: 0.8726 - val_loss: 0.3221 - val_accuracy: 0.8686


We can see that the accuracy improved significantly after each epoch achieving around 90% of accuracy at the end.

In [11]:
print(fit_result.history['accuracy'])

[0.6274461, 0.7664237, 0.82253397, 0.8529353, 0.87260383]


We can now save our trained model so we can load it and use without the need for it to be trained again in the future.

In [12]:
model.save_weights('model.h5')
# model.load_weights('model.h5')

Testing time

In [181]:
test_img_gen.reset()

pred = model.predict_generator(test_img_gen, steps=test_total/batch_size, verbose=1)

predicted_class_indices = np.round(pred)

labels = (train_img_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k[0]] for k in predicted_class_indices]

filenames = test_img_gen.filenames
results = pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

correct = 0
incorrect = 0
for index, value in results.iterrows():
    filename = value[0].replace('cats_and_dogs/', '')
    prediction_value = value[1]
    if (filename.split('.')[0] + 's' == prediction_value):
        correct = correct + 1
    else:
        incorrect = incorrect + 1

print('Accuracy on the test data: ' + str(round((correct/test_total)*100, 1)) + '%')

7/6 [=================================] - 3s 421ms/step
Accuracy on the test data: 90.0%
